# Geohash + CID


In [ ]:
#!pip install pygeohash

In [1]:
import pygeohash as pgh
##load geojson
import geopandas as gpd
import pandas as pd
import os
import sys
sys.path.append("../")

Can only import/use native python functions.


In [ ]:
from geohashtree.geohashtree import LiteTreeOffset, LiteTreeCID, FullTreeFile
from geohashtree.filesystem import ipfs_add_feature,ipfs_add_index_folder

In [2]:
asset = "dc_restaurants"
geojson_path = f"../assets/{asset}.geojson"
index_path = f"../data/geohash_offset_{asset}/index"

## Detached tree with offset
### Index Preparation
using one large file + lite index tree with offset

In [ ]:
offsettree = LiteTreeOffset()
offsettree.add_from_geojson(geojson_path)
offsettree.generate_tree_index(index_path)

### Onloading

In [ ]:
ipfs_add_index_folder(index_path)

In [5]:
index_cid = "bafybeib6ex2onm2wk5wxrdfe73l4m3la3c35nr5f4b4trbtidqaofduqiu"

In [6]:
ipfs_add_feature(geojson_path)

'bafybeia7jlkyzt22qgh66felldr322l5c7jdvm4xspr222f4usnd2ttbze'

In [ ]:
cid = "bafybeia7jlkyzt22qgh66felldr322l5c7jdvm4xspr222f4usnd2ttbze"

In [ ]:
##debugging
# import subprocess

# of,lg = (6266126, 4005)

# result = subprocess.run(['ipfs', 'cat', cid, "-o",str(of),"-l",str(lg)], stdout=subprocess.PIPE, check=True, text=True)

## Spatial query
Perform a spatial query and return a list of geohashes

In [3]:
geohashes = ['dqcjy',
 'dqcjz',
 'dqcjw',
 'dqcjv',
 'dqcjx',
 'dqcjq',
 'dqcjt',
 'dqcju',
 'dqcjr',
 'dqcjm',
 'dqcjs',
 'dqcjg',
 'dqcjk',
 'dqcnn',
 'dqcnq',
 'dqcnp',
 'dqcnj',
 'dqcnr',
 'dqcnm',
 'dqcnh',
 'dqcnk',
 'dqcmb',
 'dqcm8',
 'dqcm2',
 'dqcq0']

In [ ]:
geohashes = ['dqcnn', 'dqcjz', 'dqcjw', 'dqcjv', 'dqcnp', 'dqcjx', 'dqcjt', 'dqcnj']

## Feature Retrieval

In [ ]:
offsettree.query(geohashes,index_cid)

In [ ]:
offsettree.retrieve(geohashes,index_cid)

## Detached tree with CID

In [1]:
import sys
sys.path.append("../")
from geohashtree.geohashtree import LiteTreeCID
from geohashtree.filesystem import ipfs_add_index_folder,ipfs_get_index_folder

Can only import/use native python functions.


IPFS path: /usr/local/bin/ipfs


In [13]:
#Entire workflow for LiteTreeCID

asset = "dc_restaurants"
raw_path = f"../assets/{asset}.parquet"
index_path = f"../data/geohash_cid_{asset}_parq/index"
pieces_path = f"../data/geohash_cid_{asset}_parq"

In [14]:
cidtree = LiteTreeCID()

In [16]:
cidtree.add_from_parquet(raw_path,pieces_path,precision=4)

bafkreidem5suuztpntgpz5pgui7ogx7qifsehp5xjfyuqremxu376mn2du
bafkreigv4hqwvw3tmswgk7b2kw6ppeiwp7okzytpnf26ipugeifokzw4ly
bafkreifx7aepqoh7b6sb7xyo2sivanvawtqs7w4abgku5uqik3d5ebz4ge
bafkreievembse7sxbatctkx2yck7h4gy7srgi7dlbpuzh66wajv4bpt5iu
bafybeigy2g7xwfrbfd2jkxnorhr4iutxhuhamfjyztlsfbs77qxltbuwke
bafkreicb32q6lzsxi5ocn2muc2w2dezawmoffq4at7pwglbyxqby7u6eu4
bafkreiheon2kwwaopsxfokeiq6megbuq44z2wt7r6url4faxn7cacafgda
bafkreiadohbld422sbtnirafw3zq4mhxdpql2ykwdpqpqussswb4qnqxse
bafkreih2soe5iqn5goz5qglvkm7isd4e5jyx666a2tigrgcndnz2474sv4
bafkreide5jcydthejgzydj7tu6sbsm3e6znmix37rzuybd6uxvkbqidvr4


In [17]:
cidtree.generate_tree_index(index_path)

In [4]:
cidtree.upload_parquet_files(pieces_path)

In [18]:
ipfs_add_index_folder(index_path)

'bafybeihfj2qf7c4lqbwdle7e6vqclexrvc4xf3evaa77z43gu5tbtrj53e'

In [19]:
ipfs_get_index_folder('bafybeihfj2qf7c4lqbwdle7e6vqclexrvc4xf3evaa77z43gu5tbtrj53e','../data/test/geohash_cid_dc_parq')

 1.30 KiB / 1.30 KiB  100.00% 0s


'Saving file(s) to ../data/test/geohash_cid_dc_parq\n'

In [4]:
local_index_path = '../data/test/geohash_cid_dc_parq'

In [6]:
cidtree.file_format

AttributeError: 'LiteTreeCID' object has no attribute 'file_format'

In [ ]:
cidtree.

In [ ]:
import glob
piece_files = glob.glob(pieces_path+"/*.geojson")

In [ ]:
ipfs_add_feature(piece_files)

In [ ]:
geojson_path

In [5]:
geohashes = ['dqcn', 'dqcj']

In [6]:
cids = cidtree.query(geohashes,local_index_path)

In [8]:
len(cids)

2

In [10]:
cidtree.file_format = 'parquet'

In [11]:
ret = cidtree.retrieve(geohashes,local_index_path)

parquet


In [12]:
ret

,full_id,osm_id,osm_type,amenity,atm,opening_hours:bar,survey:date,brewery,payment:cheque,tourism,...,addr:street,addr:state,addr:postcode,addr:housenumber,addr:city,entrance,geometry,x,y,geohash
7,n268777767,268777767,node,restaurant,None,None,None,None,None,None,...,None,None,None,None,None,None,POINT (-77.15439 39.08388),-77.154388,39.083883,dqcn
11,n341582156,341582156,node,restaurant,None,None,None,None,None,None,...,Crabbs Branch Way,MD,20855,16811,Derwood,None,POINT (-77.16691 39.13142),-77.166907,39.131420,dqcn
38,n512951175,512951175,node,restaurant,None,None,None,None,None,None,...,Columbia Pike,MD,None,10721,None,None,POINT (-77.00405 39.03095),-77.004055,39.030949,dqcn
39,n558678266,558678266,node,restaurant,None,None,None,None,None,None,...,Rockville Pike,MD,20852,12147,Rockville,None,POINT (-77.11994 39.05788),-77.119941,39.057878,dqcn
64,n582790575,582790575,node,restaurant,None,None,None,None,None,None,...,None,None,None,None,None,None,POINT (-77.07726 39.08132),-77.077265,39.081323,dqcn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2669,n11265544918,11265544918,node,restaurant,None,None,None,None,None,None,...,14th Street Northwest,None,None,3310,None,None,POINT (-77.03292 38.93071),-77.032923,38.930711,dqcj
2670,n11275845269,11275845269,node,restaurant,None,None,None,None,None,None,...,Arlington Boulevard,VA,22042,7236,None,None,POINT (-77.19543 38.86694),-77.195429,38.866937,dqcj
2672,n11285160608,11285160608,node,restaurant,None,None,None,None,None,None,...,New Jersey Avenue Northwest,None,None,1545,None,None,POINT (-77.01643 38.91095),-77.016429,38.910949,dqcj
2673,n11286073597,11286073597,node,restaurant,None,None,None,None,None,None,...,K Street Northwest,None,None,1108,None,None,POINT (-77.02754 38.90226),-77.027543,38.902256,dqcj


In [ ]:
from geohashtree.geohashtree import FullTreeFile

## Attached tree with file

In [ ]:
#Entire workflow for FullTreeFile

asset = "us_places_100k"
geojson_path = f"../data/overture/{asset}.geojson"
index_path = f"../data/geohash_full_{asset}/index"

In [ ]:
fulltree = FullTreeFile()
fulltree.add_from_geojson(geojson_path)
fulltree.generate_tree_index(index_path)
ipfs_add_index_folder(index_path)

In [3]:
geohashes = ['dqcnn', 'dqcjz', 'dqcjw', 'dqcjv', 'dqcnp', 'dqcjx', 'dqcjt', 'dqcnj']

In [ ]:
cid = "bafybeiez73pdnpevvoptzft54zmljj54di2pp44slk67wfeo7uz5hdbqoe/d/dq/dqc/dqcn/dqcnn"

In [4]:
import os
os.path.join(cid,"dqcnn0.geojson")

NameError: name 'cid' is not defined

In [5]:
index_cid = "bafybeiez73pdnpevvoptzft54zmljj54di2pp44slk67wfeo7uz5hdbqoe"

In [6]:
results = fulltree.query(geohashes,index_cid)

In [7]:
len(results)

329

In [8]:
results

,full_id,osm_id,osm_type,amenity,atm,opening_hours:bar,survey:date,brewery,payment:cheque,tourism,...,addr:street,addr:state,addr:postcode,addr:housenumber,addr:city,entrance,x,y,geohash,geometry
0,n1366294218,1366294218,node,restaurant,None,None,None,None,None,None,...,Connecticut Avenue,MD,20895,10508,Kensington,None,-77.077347,39.028378,dqcnn0,POINT (-77.07735 39.02838)
1,n5613751173,5613751173,node,restaurant,None,None,None,None,None,None,...,Armory Avenue,MD,20895,10417,Kensington,None,-77.074518,39.027431,dqcnn0,POINT (-77.07452 39.02743)
2,n7189139268,7189139268,node,restaurant,None,None,None,None,None,None,...,Metropolitan Avenue,MD,20895,10509,Kensington,None,-77.071348,39.027516,dqcnn0,POINT (-77.07135 39.02752)
3,n8402911162,8402911162,node,restaurant,None,None,None,None,None,None,...,Kensington Parkway,MD,20895,10301,Kensington,None,-77.070937,39.025357,dqcnn0,POINT (-77.07094 39.02536)
4,n8504662317,8504662317,node,restaurant,None,None,None,None,None,None,...,Howard Avenue,MD,20895,3784,Kensington,None,-77.074266,39.027745,dqcnn0,POINT (-77.07427 39.02775)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,n6348525382,6348525382,node,restaurant,None,None,None,None,None,None,...,Twinbrook Parkway,MD,20852,12303,Rockville,None,-77.119616,39.058517,dqcnjn,POINT (-77.11962 39.05852)
8,n8735325466,8735325466,node,restaurant,None,None,None,None,None,None,...,Rockville Pike,None,None,12668,None,None,-77.122540,39.057150,dqcnjn,POINT (-77.12254 39.05715)
0,n11113180851,11113180851,node,restaurant,None,None,None,None,None,None,...,Wilkins Court,None,None,12307,None,None,-77.112674,39.057823,dqcnjq,POINT (-77.11267 39.05782)
0,n633618682,633618682,node,restaurant,None,None,None,None,None,None,...,None,None,None,None,None,None,-77.099084,39.046104,dqcnjs,POINT (-77.09908 39.04610)


In [9]:
ret = fulltree.retrieve(geohashes,index_cid)

In [10]:
ret

,full_id,osm_id,osm_type,amenity,atm,opening_hours:bar,survey:date,brewery,payment:cheque,tourism,...,addr:street,addr:state,addr:postcode,addr:housenumber,addr:city,entrance,x,y,geohash,geometry
0,n1366294218,1366294218,node,restaurant,None,None,None,None,None,None,...,Connecticut Avenue,MD,20895,10508,Kensington,None,-77.077347,39.028378,dqcnn0,POINT (-77.07735 39.02838)
1,n5613751173,5613751173,node,restaurant,None,None,None,None,None,None,...,Armory Avenue,MD,20895,10417,Kensington,None,-77.074518,39.027431,dqcnn0,POINT (-77.07452 39.02743)
2,n7189139268,7189139268,node,restaurant,None,None,None,None,None,None,...,Metropolitan Avenue,MD,20895,10509,Kensington,None,-77.071348,39.027516,dqcnn0,POINT (-77.07135 39.02752)
3,n8402911162,8402911162,node,restaurant,None,None,None,None,None,None,...,Kensington Parkway,MD,20895,10301,Kensington,None,-77.070937,39.025357,dqcnn0,POINT (-77.07094 39.02536)
4,n8504662317,8504662317,node,restaurant,None,None,None,None,None,None,...,Howard Avenue,MD,20895,3784,Kensington,None,-77.074266,39.027745,dqcnn0,POINT (-77.07427 39.02775)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,n6348525382,6348525382,node,restaurant,None,None,None,None,None,None,...,Twinbrook Parkway,MD,20852,12303,Rockville,None,-77.119616,39.058517,dqcnjn,POINT (-77.11962 39.05852)
8,n8735325466,8735325466,node,restaurant,None,None,None,None,None,None,...,Rockville Pike,None,None,12668,None,None,-77.122540,39.057150,dqcnjn,POINT (-77.12254 39.05715)
0,n11113180851,11113180851,node,restaurant,None,None,None,None,None,None,...,Wilkins Court,None,None,12307,None,None,-77.112674,39.057823,dqcnjq,POINT (-77.11267 39.05782)
0,n633618682,633618682,node,restaurant,None,None,None,None,None,None,...,None,None,None,None,None,None,-77.099084,39.046104,dqcnjs,POINT (-77.09908 39.04610)
